In [ ]:
RADIUS_ARCSEC = 0.2

NICE_CSV = b"""ra,dec,Period
94.95546,-24.73952,0.12095
95.30235,-25.27431,0.12248
94.91626,-24.69648,0.12038
95.12418,-25.04329,0.23554
58.83506,-48.79122,0.56335
94.92264,-25.23185,0.07672
94.72086,-25.05767,0.17559
94.97073,-25.13643,0.12048
59.12997,-48.78522,0.11628
94.72086,-25.05767,0.17554
"""

In [ ]:
# %pip install -U lsdb nested-pandas nested-dask hats 'dask[complete]'

In [ ]:
from functools import partial
from io import BytesIO
from pathlib import Path

import lsdb
import matplotlib.pyplot as plt
from nested_pandas import NestedDtype
import pandas as pd
from dask.distributed import Client

In [ ]:
def cast_nested(df, columns):
    return df.assign(
        **{col: df[col].astype(NestedDtype.from_pandas_arrow_dtype(df.dtypes[col])) for col in columns},
    )

In [ ]:
release = 'w_2025_07'
hats_path = Path("/sdf/data/rubin/shared/lsdb_commissioning/hats") / release

obj_catalog = lsdb.read_hats(hats_path / "object_lc").map_partitions(partial(cast_nested, columns=["forcedSource"]))
dia_catalog = lsdb.read_hats(hats_path / "diaObject_lc").map_partitions(partial(cast_nested, columns=["diaSource", "diaForcedSource"]))

nice_df = pd.read_csv(BytesIO(NICE_CSV)).reset_index()
nice_catalog = lsdb.from_dataframe(nice_df)
nice_catalog

In [ ]:
with Client(n_workers=20, threads_per_worker=1) as client:
    nice_obj = nice_catalog.crossmatch(obj_catalog, radius_arcsec=RADIUS_ARCSEC, suffixes=["_nice", ""]).compute()
    nice_dia = nice_catalog.crossmatch(dia_catalog, radius_arcsec=RADIUS_ARCSEC, suffixes=["_nice", ""]).compute()
print(nice_obj.shape, nice_dia.shape)
df = pd.merge(nice_obj, nice_dia, on="index_nice", suffixes=["_obj", "_dia"])

In [ ]:
COLORS = {
    "u": "#56b4e9",
    "g": "#009e73",
    "r": "#f0e442",
    "i": "#cc79a7",
    "z": "#d55e00",
    "y": "#0072b2",
}


def plot(ax, lc, x_name="midpointMjdTai", x_label="MJD"):
    lc = lc.query(
        "~psfFlux_flag"
        # " and ~pixelFlags_suspect"
        " and ~pixelFlags_saturated"
        " and ~pixelFlags_cr"
        " and ~pixelFlags_bad"
    )

    for band, color in COLORS.items():
        band_lc = lc.query(f"band == '{band}'")
        ax.errorbar(
            band_lc[x_name], band_lc.psfFlux, band_lc.psfFluxErr,
            fmt='o', label=band, color=color, alpha=0.5, markersize=4,
        )
    ax.set_xlabel(x_label)
    ax.set_ylabel("Flux, nJy")
    ax.legend(loc="lower right")

for _, row in df.iterrows():
    fig, ax = plt.subplots(3, 2, figsize=(10, 10))
    for col in range(2):  # Loop over columns
        for plot_row in range(1, 3):  # Start from second row
            ax[plot_row, col].sharex(ax[0, col])  # Share x-axis with the first row of the same column
    fig.suptitle(f"RA={row.ra_nice_obj:.5f}, Dec={row.dec_nice_obj:.5f}")
    for ax_, lc, title in zip(ax, [row.diaSource, row.diaForcedSource, row.forcedSource], ["Dia Source", "Dia Forced Source", "Forced Source"]):
        ax_[0].set_title(title)
        lc = lc.assign(phase=(lc.midpointMjdTai - lc.midpointMjdTai.loc[lc.psfFlux.idxmax()]) % row.Period_nice_obj / row.Period_nice_obj)
        plot(ax_[0], lc, x_name="midpointMjdTai", x_label="MJD")
        plot(ax_[1], lc, x_name="phase", x_label="phase")
    plt.tight_layout()
    plt.show()

In [ ]:
row.forcedSource